In [15]:
import tabula #Extracting tables from pdf
#from pdfminer.high_level import extract_pages, ectract_text
import os
import re
from tika import parser # pip install tika

import pandas as pd
import numpy as np

In [16]:
def export_to_csv(data_frame, file_name):
    """
    Export a DataFrame to a CSV file.

    Parameters:
        - data_frame: pandas DataFrame
        - file_name: str, name of the CSV file (without the extension)
    """
    #print date stamp - datetime.datetime.now()
    
    #t_stamp = str(datetime.datetime.now()).split(".")[0].replace(" ","_")
    
    file_path = f"{file_name}.csv"
    
    data_frame.to_csv(file_path, index=False)
    print(f"DataFrame successfully exported to {file_path}")

In [17]:
path = os.listdir("/Users/Moni/Ironhack/my_ironhack/projects/final-project/data/historico-facturas-pdf/family")

# Extract name of documents
files = [i for i in path if ".DS" not in i]

#for testing
#files = files[3:5]

In [18]:
df = pd.DataFrame(columns=['description', 'units', 'price', "date", "order_num"])

path = "../../data/historico-facturas-pdf/family/"

for pdf in files:
    
    order_num = pdf.split("-")[1].replace(".pdf", "")
    
    #parser the text in the pdf (returns a dict)
    raw = parser.from_file(f"{path}{pdf}")
    
    # Extract the content (returns a str)
    content = raw['content']
    
    # He know there is only one date in the document (and the format)
    pattern = r'\b\d{2}/\d{2}/\d{2}\b'  #meets the date format
    
    # Extract the date - returns a list of one elemnt
    date = re.findall(pattern, content)[0]
    
    #Extract tables from the pdf:
    tables = tabula.read_pdf(f"{path}{pdf}", pages="all", pandas_options={'header': None})
    
    #We know last table is always resume of ticket.exclude:
    #tables = tables[:-1]
    
    #Information about products(main table) in different pages, are extracted as different tables:
    for table in tables:
        
        if table[0][0] == "IVA":
            continue
        
        table.rename(columns={0: "description", 1:"units", 2:"price"}, inplace=True)
        table["date"] = date
        table["order_num"] = order_num

        result = pd.concat([df, table])

        df = result
print("pdf tables extrated")

pdf tables extrated


In [19]:
df

,description,units,price,date,order_num,3
0,Jamón serrano Incarlopsa lonchas,1,"3,49 €",01/10/20,302316,NaN
1,Queso fresco de cabra Hacendado,1,"2,20 €",01/10/20,302316,NaN
2,Queso rallado 4 quesos Hacendado,1,"1,59 €",01/10/20,302316,NaN
3,Perlas de mozzarella fresca Hacendado,1,"1,70 €",01/10/20,302316,NaN
4,Mozzarella di bufala campana Hacendado,1,"1,70 €",01/10/20,302316,NaN
...,...,...,...,...,...,...
4,Queso curado Entrepinares,1,"2,57 €",19/06/19,207666,NaN
5,Morcilla oreada d' Ontinyent,1,"1,39 €",19/06/19,207666,NaN
6,Solomillo de cerdo,1,"4,32 €",19/06/19,207666,NaN
7,Chorizo oreado,1,"1,94 €",19/06/19,207666,NaN


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 840 entries, 0 to 8
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   description  827 non-null    object
 1   units        765 non-null    object
 2   price        816 non-null    object
 3   date         840 non-null    object
 4   order_num    840 non-null    object
 5   3            51 non-null     object
dtypes: object(6)
memory usage: 45.9+ KB


In [21]:
def drop_na(df, column):
    nans = df[df[column].isna()].shape[0]
    df.dropna(subset=['description'], inplace = True)
    
    print(f"{nans} rows droped in colum {column}")
    
    return

In [22]:
drop_na(df, "description")

13 rows droped in colum description


In [23]:
# Vamos a ver que datos podemos encontrar en la columna 3:
not_nan_3 = df[df[3].isna() == False]

not_nan_3.sample()

,description,units,price,date,order_num,3
31,Tomates cherry,NaN,1.0,13/07/22,996926,"1,65 €"


In [24]:
#Eliminamos estas filas (luego las vamos a volver a incluir)
df = df[df[3].isna() == True]
df.shape

(779, 6)

In [25]:
# Vemos que es como si se hubieram desplazado, lo reognanizamos: 
not_nan_correct = not_nan_3
not_nan_correct["units"] = not_nan_3["price"]
not_nan_correct["price"] = not_nan_3[3]
not_nan_correct[3] = np.nan

/var/folders/k4/q8g97p495cx7f63kbs0wzyc00000gn/T/ipykernel_12178/4124051874.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_nan_correct["units"] = not_nan_3["price"]
/var/folders/k4/q8g97p495cx7f63kbs0wzyc00000gn/T/ipykernel_12178/4124051874.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_nan_correct["price"] = not_nan_3[3]
/var/folders/k4/q8g97p495cx7f63kbs0wzyc00000gn/T/ipykernel_12178/4124051874.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [26]:
not_nan_correct.sample()
df = pd.concat([df, not_nan_correct])

In [27]:
df.shape # Se mantien el numpero de columnas por lo que lo hemos hecho bien

(827, 6)

In [28]:
df = df.drop(3, axis=1)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 827 entries, 0 to 18
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   description  827 non-null    object
 1   units        803 non-null    object
 2   price        803 non-null    object
 3   date         827 non-null    object
 4   order_num    827 non-null    object
dtypes: object(5)
memory usage: 38.8+ KB


In [30]:
df.dropna(subset=['units'], inplace=True)

In [31]:
df

,description,units,price,date,order_num
0,Jamón serrano Incarlopsa lonchas,1,"3,49 €",01/10/20,302316
1,Queso fresco de cabra Hacendado,1,"2,20 €",01/10/20,302316
2,Queso rallado 4 quesos Hacendado,1,"1,59 €",01/10/20,302316
3,Perlas de mozzarella fresca Hacendado,1,"1,70 €",01/10/20,302316
4,Mozzarella di bufala campana Hacendado,1,"1,70 €",01/10/20,302316
...,...,...,...,...,...
14,Contramuslos de pollo deshuesados y sin piel,1.0,"4,28 €",19/10/21,337881
15,Solomillos de pollo,1.0,"5,19 €",19/10/21,337881
16,Filetes de dorada,1.0,"6,77 €",19/10/21,337881
17,Aguacates,1.0,"2,13 €",19/10/21,337881


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 803 entries, 0 to 18
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   description  803 non-null    object
 1   units        803 non-null    object
 2   price        803 non-null    object
 3   date         803 non-null    object
 4   order_num    803 non-null    object
dtypes: object(5)
memory usage: 37.6+ KB


In [33]:
# Last tranformatios for casting and uniformity 

In [34]:
df["price_eu"] = df["price"].str.replace("€","").str.replace(",",".").str.strip().astype(float)
df["unit_price"] = df["price_eu"]/df["units"]

In [35]:
df["units"] = df["units"].astype(int)

In [36]:
df = df[["description", "units", "price_eu", "unit_price", "date", "order_num"]]

In [37]:
df

,description,units,price_eu,unit_price,date,order_num
0,Jamón serrano Incarlopsa lonchas,1,3.49,3.49,01/10/20,302316
1,Queso fresco de cabra Hacendado,1,2.20,2.2,01/10/20,302316
2,Queso rallado 4 quesos Hacendado,1,1.59,1.59,01/10/20,302316
3,Perlas de mozzarella fresca Hacendado,1,1.70,1.7,01/10/20,302316
4,Mozzarella di bufala campana Hacendado,1,1.70,1.7,01/10/20,302316
...,...,...,...,...,...,...
14,Contramuslos de pollo deshuesados y sin piel,1,4.28,4.28,19/10/21,337881
15,Solomillos de pollo,1,5.19,5.19,19/10/21,337881
16,Filetes de dorada,1,6.77,6.77,19/10/21,337881
17,Aguacates,1,2.13,2.13,19/10/21,337881


In [40]:
df['date'] = pd.to_datetime(df['date'])

/var/folders/k4/q8g97p495cx7f63kbs0wzyc00000gn/T/ipykernel_12178/3532345252.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'])
/var/folders/k4/q8g97p495cx7f63kbs0wzyc00000gn/T/ipykernel_12178/3532345252.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date'])


In [42]:
export_to_csv(df,"../../data/history-pruchase-family")

DataFrame successfully exported to ../data/history-pruchase-family.csv


In [41]:
df

,description,units,price_eu,unit_price,date,order_num
0,Jamón serrano Incarlopsa lonchas,1,3.49,3.49,2020-01-10,302316
1,Queso fresco de cabra Hacendado,1,2.20,2.2,2020-01-10,302316
2,Queso rallado 4 quesos Hacendado,1,1.59,1.59,2020-01-10,302316
3,Perlas de mozzarella fresca Hacendado,1,1.70,1.7,2020-01-10,302316
4,Mozzarella di bufala campana Hacendado,1,1.70,1.7,2020-01-10,302316
...,...,...,...,...,...,...
14,Contramuslos de pollo deshuesados y sin piel,1,4.28,4.28,2021-10-19,337881
15,Solomillos de pollo,1,5.19,5.19,2021-10-19,337881
16,Filetes de dorada,1,6.77,6.77,2021-10-19,337881
17,Aguacates,1,2.13,2.13,2021-10-19,337881
